In [2]:
import pandas as pd
import chardet
import mysql.connector as db
import sqlalchemy
from sqlalchemy import create_engine

In [7]:
mydb = db.connect(
    host="localhost",
    user="root",
    password="root",
    database="global_electronics"
)
mycursor = mydb.cursor(buffered=True)

mycursor.execute("USE global_electronics")

-------------------------------

1) Most sold Item Over the years with SOLD QTY and OVERALL PROFIT?

In [8]:
query1 = """SELECT 
    o.ProductKey,
    p.ProductName, 
    SUM(o.Quantity) AS TotalQuantity,
    FORMAT(SUM((p.UnitPriceUSD - p.UnitCostUSD) * o.Quantity), 2) AS 'Profit USD'
FROM 
    orders o
JOIN 
    Products p ON o.ProductKey = p.ProductKey
GROUP BY 
    p.ProductName, o.ProductKey
ORDER BY 
    TotalQuantity DESC
LIMIT 10;"""

mycursor.execute(query1)
df_1 = pd.DataFrame(mycursor.fetchall(),columns= ['Product Id', 'Product Name', 'Total QTY Sold','Total Profit USD'])
#df_1['Total_amount'] = df_1['Total_amount'].round(2)
df_1.index += 1
df_1

,Product Id,Product Name,Total QTY Sold,Total Profit USD
1,444,WWI Desktop PC2.33 X2330 Black,550,"337,985.99"
2,458,WWI Desktop PC1.80 E1800 White,538,"60,627.22"
3,434,Adventure Works Desktop PC2.30 MD230 White,521,"168,564.34"
4,424,Adventure Works Desktop PC1.60 ED160 Black,521,"68,938.72"
5,425,Adventure Works Desktop PC1.80 ED180 Black,520,"94,052.40"
6,423,Adventure Works Desktop PC2.30 MD230 Black,514,"166,299.56"
7,446,WWI Desktop PC1.60 E1600 Black,509,"54,875.29"
8,440,WWI Desktop PC1.60 E1600 Silver,507,"54,659.67"
9,435,Adventure Works Desktop PC1.60 ED160 White,505,"66,821.60"
10,448,WWI Desktop PC1.80 E1801 Black,505,"66,811.49"


----------------------------

2) Top 5 stores wich perform good sales every year

In [52]:
query2 = """SELECT 
    storekey, 
    YEAR(orderdate) AS year,
    FORMAT(SUM(total_price), 2) AS total_sales
FROM 
    (
        SELECT 
            o.storekey,
            SUM(p.UnitPriceUSD * o.Quantity) AS total_price,
            o.orderdate
        FROM 
            orders o
        JOIN 
            Products p ON o.ProductKey = p.ProductKey
        GROUP BY 
            o.storekey, o.ProductKey, o.orderdate
    ) AS subquery
GROUP BY 
    storekey, YEAR(orderdate)
ORDER BY 
    storekey;"""

mycursor.execute(query2)
df_2 = pd.DataFrame(mycursor.fetchall(),columns= ['Store_Id', 'Year', 'Total_Sales'])
# Pivot the DataFrame
df_2_final = df_2.pivot(index='Store_Id', columns='Year', values='Total_Sales')

# Reset the index to make 'Store_Id' a column again
df_2_final = df_2_final.reset_index()

# Rename the columns (optional, the pivot table will have the correct names by default)
df_2_final.columns.name = None  # Remove the name of the columns axis
df_2_final = df_2_final.fillna(0.0)

df_2_final.index += 1
# Display the resulting DataFrame
df_2_final

,Store_Id,2016,2017,2018,2019,2020,2021
1,0,"1,169,315.86","1,388,497.44","2,582,287.13","3,908,495.70","2,069,950.90","285,777.60"
2,1,"16,325.43","24,980.24","54,196.31","83,345.18","59,547.84","4,634.93"
3,2,"15,175.99",0.0,0.0,0.0,0.0,0.0
4,4,"33,116.81","72,686.25","116,030.59","181,319.47","34,941.02","4,380.88"
5,5,"127,675.35","89,379.16","235,314.03","277,555.37","115,533.80","14,220.48"
6,6,"81,516.32","94,777.30","103,192.69","146,542.80","111,147.00","1,605.83"
7,8,"179,337.58","195,369.79","323,814.43","344,169.46","160,783.21","28,751.46"
8,9,"212,341.79","183,846.48","281,824.16","433,627.51","194,642.34","29,867.78"
9,10,"117,234.51","126,977.82","241,664.53","351,148.61","196,229.14","9,931.19"
10,12,"14,444.58","16,674.03","40,924.45","53,193.33","54,126.12","3,728.53"


---------------------------

3) SORT OUT THE CUSTOMER AGE ON PURCHASING

In [53]:
query3 = """SELECT
    age_bucket,
    COUNT(*) AS count
FROM (
    SELECT
        c.CustomerKey,
        o.Orderdate,
        c.birthday,
        CASE
            WHEN YEAR(STR_TO_DATE(o.Orderdate, '%Y-%m-%d')) - YEAR(STR_TO_DATE(c.birthday, '%Y-%m-%d')) < 18 THEN '<=18'
            WHEN YEAR(STR_TO_DATE(o.Orderdate, '%Y-%m-%d')) - YEAR(STR_TO_DATE(c.birthday, '%Y-%m-%d')) BETWEEN 18 AND 24 THEN '18-24'
            WHEN YEAR(STR_TO_DATE(o.Orderdate, '%Y-%m-%d')) - YEAR(STR_TO_DATE(c.birthday, '%Y-%m-%d')) BETWEEN 25 AND 34 THEN '25-34'
            WHEN YEAR(STR_TO_DATE(o.Orderdate, '%Y-%m-%d')) - YEAR(STR_TO_DATE(c.birthday, '%Y-%m-%d')) BETWEEN 35 AND 44 THEN '35-44'
            WHEN YEAR(STR_TO_DATE(o.Orderdate, '%Y-%m-%d')) - YEAR(STR_TO_DATE(c.birthday, '%Y-%m-%d')) BETWEEN 45 AND 54 THEN '45-54'
            WHEN YEAR(STR_TO_DATE(o.Orderdate, '%Y-%m-%d')) - YEAR(STR_TO_DATE(c.birthday, '%Y-%m-%d')) BETWEEN 55 AND 64 THEN '55-64'
            ELSE '>65'
        END AS age_bucket
    FROM CUSTOMERS AS c
    JOIN ORDERS o ON c.CustomerKey = o.CustomerKey
) AS age_groups
GROUP BY age_bucket;"""

mycursor.execute(query3)
df_3 = pd.DataFrame(mycursor.fetchall(),columns= ['AGE RANGE', 'COUNT'])

df_3.index += 1
df_3

,AGE RANGE,COUNT
1,45-54,9420
2,18-24,6561
3,35-44,9371
4,>65,17831
5,25-34,9386
6,55-64,9335
7,<=18,980


-----------------------------

4) GENDER COUNT

In [55]:
query4 = """SELECT gender, COUNT(*) AS Count_gender
FROM customers
GROUP BY gender;"""

mycursor.execute(query4)
df_4 = pd.DataFrame(mycursor.fetchall(),columns= ['GENDER', 'COUNT'])

df_4.index += 1
df_4

,GENDER,COUNT
1,Female,7518
2,Male,7748


---------------------------

5) CUSTOMER COUNT COUNTRY WISE

In [59]:
query5 = """SELECT 
    continent,country,
    COUNT(CustomerKey) AS customer_count
FROM 
    CUSTOMERS
GROUP BY 
    continent,country
ORDER BY 
    customer_count DESC"""

mycursor.execute(query5)
df_5 = pd.DataFrame(mycursor.fetchall(),columns= ['CONTINENT', 'COUNTRY','CUSTOMER COUNT'])

df_5.index += 1
df_5

,CONTINENT,COUNTRY,CUSTOMER COUNT
1,North America,United States,6828
2,Europe,United Kingdom,1944
3,North America,Canada,1553
4,Europe,Germany,1473
5,Australia,Australia,1420
6,Europe,Netherlands,733
7,Europe,France,670
8,Europe,Italy,645


-------------------------------

6) MOST SOLD PRODUCTS CATAGORY WISE

In [60]:
query6 = """SELECT 
    p.Category,
    p.Subcategory,
    SUM(o.Quantity) AS qty
FROM 
    orders o
JOIN 
    Products p ON o.ProductKey = p.ProductKey
GROUP BY 
    p.Category, p.Subcategory
ORDER BY 
    qty DESC
LIMIT 10;"""

mycursor.execute(query6)
df_6 = pd.DataFrame(mycursor.fetchall(),columns= ['Category', 'Subcategory','Total QTY Sold'])

df_6.index += 1
df_6

,Category,Subcategory,Total QTY Sold
1,"Music, Movies and Audio Books",Movie DVD,28802
2,Computers,Desktops,20626
3,Audio,Bluetooth Headphones,13653
4,Games and Toys,Download Games,11547
5,Games and Toys,Boxed Games,11044
6,Cell phones,Touch Screen Phones,10630
7,Cell phones,Smart phones & PDAs,10077
8,Audio,Recording Pen,6617
9,TV and Video,Televisions,5625
10,Cell phones,Home & Office Phones,5432


----------------------------

7) LEAST SOLD PRODUCTS CATOGORY WISE

In [61]:
query7 = """SELECT 
    p.Category,
    p.Subcategory,
    SUM(o.Quantity) AS qty
FROM 
    orders o
JOIN 
    Products p ON o.ProductKey = p.ProductKey
GROUP BY 
    p.Category, p.Subcategory
ORDER BY 
    qty ASC
LIMIT 10;"""

mycursor.execute(query7)
df_7 = pd.DataFrame(mycursor.fetchall(),columns= ['Category', 'Subcategory','Total QTY Sold'])

df_7.index += 1
df_7

,Category,Subcategory,Total QTY Sold
1,Home Appliances,Air Conditioners,884
2,Home Appliances,Lamps,1164
3,Home Appliances,Washers & Dryers,1180
4,Home Appliances,Fans,1241
5,TV and Video,Car Video,1363
6,TV and Video,Home Theater System,1375
7,Home Appliances,Coffee Machines,2295
8,Home Appliances,Refrigerators,2324
9,TV and Video,VCD & DVD,2873
10,Audio,MP4&MP3,3220


----------------------

8) comparing online sales and physical store sales

In [68]:
query8 = """SELECT
    CASE
        WHEN o.storekey = 0 THEN 'ONLINE STORE'
        ELSE 'PHYSICAL STORES'
    END AS store_group,
    SUM(o.Quantity) AS total_quantity,
    FORMAT(SUM(p.UnitPriceUSD * o.Quantity),2) AS total_sales
FROM
    orders o
JOIN
    Products p ON o.ProductKey = p.ProductKey
GROUP BY
    store_group
ORDER BY
    store_group;"""

mycursor.execute(query8)
df_8 = pd.DataFrame(mycursor.fetchall(),columns= ['STORE TYPE', 'Total QTY Sold', 'TOTAL SALES USD'])

df_8.index += 1
df_8

,STORE TYPE,Total QTY Sold,TOTAL SALES USD
1,ONLINE STORE,41311,"11,404,324.62"
2,PHYSICAL STORES,156446,"44,351,154.91"


In [49]:
df_2_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58 entries, 0 to 57
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Store_Id  58 non-null     int64 
 1   2016      58 non-null     object
 2   2017      58 non-null     object
 3   2018      58 non-null     object
 4   2019      58 non-null     object
 5   2020      58 non-null     object
 6   2021      58 non-null     object
dtypes: int64(1), object(6)
memory usage: 3.3+ KB


In [11]:
df_2['Year'].unique()

[np.int64(2016),
 np.int64(2017),
 np.int64(2018),
 np.int64(2019),
 np.int64(2020),
 np.int64(2021)]

In [ ]:
query = "SELECT video_info.Views as Views, video_info.Title, channel_info.Channel_Name FROM video_info join channel_info on channel_info.Channel_Id = video_info.Channel_Id ORDER BY Views desc limit 10;"
        result_3 = get_cursor(query)
        df3 = pd.DataFrame(result_3, columns=['Video Views','Video Name','Channel Name']).reset_index(drop=True)
        df3.index += 1
        st.dataframe(df3)